# Gerardo Daniel Vega Rosales

In [ ]:
%%bash
echo "Start Script"
if [ -d ".git" ]
then
echo "We can perform the git command"
git add .
git commit -m "update notebook"
echo "Pushing data to remote server!!!"
git push
else
echo "Please add the origin after perform git commands"
fi
echo "Finish Script"